<a href="https://colab.research.google.com/github/Sheha1218/AI-Powered-Driver-Drowsiness-Detection-System/blob/main/Drowsiness_Detection_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms,datasets
from torch.utils.data import random_split,DataLoader
import cv2
import numpy as np

In [3]:
od.download('https://www.kaggle.com/datasets/prasadvpatil/mrl-dataset')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: shehangunathilaka
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/prasadvpatil/mrl-dataset


100%|██████████| 22.6M/22.6M [00:00<00:00, 1.46GB/s]

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
    transforms.Resize((224,244)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10)

])

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
data =('/content/mrl-dataset/train')

In [7]:
data = datasets.ImageFolder(data,transform=transform)

In [8]:
train_data = int(0.8*len(data))
test_data =len(data)-train_data

In [9]:
train_data,test_data = random_split(data,[train_data,test_data])

In [10]:
train_loader = DataLoader(train_data,batch_size=32,shuffle=True)
test_loader=DataLoader(test_data,batch_size=32,shuffle=False)

In [11]:
image,label =train_data[0]
print(f'image shape{image.shape}')
print(f'label value {label}')

image shapetorch.Size([3, 224, 244])
label value 1


In [12]:
print(image.size())

torch.Size([3, 224, 244])


In [13]:
print(data.classes)

['Closed_Eyes', 'Open_Eyes']


In [14]:
class_names = ['Closed_Eyes', 'Open_Eyes']

In [15]:
class neuralnet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(3,12,5)
    self.pool =nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(12,24,5)


    self.fc1=nn.Linear(24*53*58,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,16)
    self.fc4 =nn.Linear(16,2)

  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=torch.flatten(x,1)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=F.relu(self.fc3(x))
    x=self.fc4(x)
    return x

In [16]:
model=neuralnet().to(device)

In [17]:
loss=nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [18]:
for epoch in range (10):
  print(f'traing epochs : {epoch}')

  running_loss=0.0
  total =0
  correct =0
  for i,data_n in enumerate(train_loader):
    inputs,labels =data_n
    inputs =inputs.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()

    output = model(inputs)

    loss_value =loss(output,labels)
    loss_value.backward()
    optimizer.step()

    running_loss+=loss_value.item()

    _,predicted =torch.max(output,1)
    total += labels.size(0)
    correct += (predicted==labels).sum().item()

  epoch_loss =running_loss /len(train_loader)
  epoch_accuracy = correct/total

  print(f'loss {epoch_loss:.4f} accuracy :{epoch_accuracy:.4f}')



traing epochs : 0
loss 0.2056 accuracy :0.9103
traing epochs : 1
loss 0.0441 accuracy :0.9866
traing epochs : 2
loss 0.0958 accuracy :0.9722
traing epochs : 3
loss 0.0266 accuracy :0.9916
traing epochs : 4
loss 0.0190 accuracy :0.9944
traing epochs : 5
loss 0.0128 accuracy :0.9969
traing epochs : 6
loss 0.0403 accuracy :0.9878
traing epochs : 7
loss 0.0103 accuracy :0.9969
traing epochs : 8
loss 0.0080 accuracy :0.9972
traing epochs : 9
loss 0.0318 accuracy :0.9903


In [19]:
torch.save(model.state_dict(),'driver.pth')